In [ ]:
%matplotlib inline

In [ ]:
import sqlite3
import numpy as np
import pandas as pd
import scipy
import pytpc
from math import ceil, floor
from pytpc.constants import *
from pytpc.cleaning import apply_clean_cut
from pytpc.tpcplot import pad_plot
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.path import Path
from matplotlib.patches import PathPatch
import matplotlib as mpl
import seaborn.apionly as sns
import glob
import re
import h5py
import yaml
import sys
if '../fitters/' not in sys.path:
    sys.path.append('../fitters/')
from pytpc.relativity import find_proton_params

In [ ]:
sns.set_context('talk')

In [ ]:
mcmin_h5path = '/Users/josh/Documents/Data/Results/e15503b/mcmin/Nov3/results.h5'
# mcmin_h5path = '/Users/josh/Documents/Code/ar40-aug15/fitters/test_results.h5'

In [ ]:
def transfer(amp, shape, offset=0):
    t = np.arange(512)
    f = amp*np.exp(-3*(t-offset)/shape)*np.sin((t-offset)/shape)*((t-offset)/shape)**3 / 0.044
    return np.where(t >= offset, f, np.zeros(t.shape))

In [ ]:
with open('../fitters/config_e15503b.yml', 'r') as f:
    config = yaml.load(f)
with open('../fitters/config_e15503b_macmini.yml', 'r') as f:
    patch = yaml.load(f)
config.update(patch)

In [ ]:
runtables = pd.read_csv('/Users/josh/Documents/Data/Meta/e15503b/e15503b-rundata-parseable.csv', index_col='run_num')

In [ ]:
vmedata = pd.read_hdf('/Users/josh/Documents/Data/Results/e15503b/select_vme/vme_good_events_e15503b.h5')
vmedata.index = ['{:04g}_{:d}'.format(r, e) for r, e in zip(vmedata.run_num, vmedata.evt_id)]

# Merge dbs

In [ ]:
rawdbs = glob.glob('/Users/josh/Documents/Data/Results/e15503b/mcmin/2016/Nov3/raw/*.db')
# rawdbs = glob.glob('../fitters/*test*.db')
# rawdbs = ['../fitters/run_0166_results.db']
print('Found {} databases'.format(len(rawdbs)))

In [ ]:
rawres = []
for db in rawdbs:
    try:
        with sqlite3.connect(db) as conn:
            df = pd.read_sql('SELECT * FROM mcmin_results', conn)
    except Exception as e:
        print('Problem with database', db, e)
        continue
    else:
        run_num = re.search(r'mcmin_run_(\d+)', db).group(1)
        df['run_num'] = int(run_num)
        df.index = ['{:04g}_{:d}'.format(r, e) for r, e in zip(df.run_num, df.evt_id)]
    rawres.append(df)
res = pd.concat(rawres, axis=0)
del rawres
del df

In [ ]:
# res.to_hdf(mcmin_h5path, 'Results')

# Import results

In [ ]:
# res = pd.read_hdf(mcmin_h5path, 'Results')

# Merge with VME data

In [ ]:
res = pd.merge(res, vmedata[['height', 'cfd_pos', 'group']], left_index=True, right_index=True)

In [ ]:
res.rename(columns={'height': 'ic_height', 'cfd_pos': 'ic_cfd_pos', 'group': 'vme_group'}, inplace=True)

In [ ]:
testdata = res[goodfitcut].sample(n=20000).copy()
testdata['npts'] = 0

for i, row in testdata.iterrows():
    fp = '/Users/josh/Documents/Data/Merged/e15503b/clean/clean_run_{:04d}.h5'.format(int(row.run_num))
    with h5py.File(fp, 'r') as f:
        ds = f['/clean/{}'.format(int(row.evt_id))]
        raw_xyz_full = ds[:]
        cx, cy = ds.attrs['center']
        
    raw_xyz = apply_clean_cut(raw_xyz_full)
    
    testdata.loc[i, 'npts'] = len(raw_xyz)

In [ ]:
d = testdata[testdata.npts > 0]
plt.scatter(d.cm_angle, d.enu0, c=d.npts, alpha=0.5, cmap='viridis', linewidth=0)
plt.colorbar()
plt.xlim(0, 2.6)
plt.ylim(0, 8)

In [ ]:
d = testdata[testdata.npts > 200]
plt.hist2d(d.cm_angle, d.enu0, bins=50, range=((0, 2.0), (0, 6)), cmap='viridis');
plt.gca().grid(False)

In [ ]:
def histerr(x, bins, **kwargs):
    xb, xe = np.histogram(x, bins=bins)
    plt.errorbar(xe[:-1] + (xe[1] - xe[0]) / 2, xb, yerr=np.sqrt(xb), fmt='b.', ecolor='b', elinewidth=1,
                 capthick=1, capsize=1, **kwargs)
#     plt.step(xe[:-1], xb, linewidth=1, where='post')
    
dtheta = 5.0
dE = 50 * 1e-3
histbins = (np.arange(0, 180 + dtheta, dtheta),
            np.arange(0, 5.0 + dE, dE))
chosen_theta = 65
    
d = testdata[testdata.npts > 0].copy()
d['thbin'] = histbins[0][np.digitize(d.cm_angle / degrees, histbins[0])]
subplotdata = d[d.thbin == chosen_theta]

histerr(subplotdata.kine_vert_en, histbins[1])
# plt.hist(subplotdata.kine_vert_en, bins=histbins[1], histtype='stepfilled')

plt.xlabel('46Ar vertex energy [MeV/u]')
plt.ylabel('Count')
plt.title(r'$\theta_{{CM}} = {}^\circ$'.format(chosen_theta))
plt.tight_layout()

# Rutherford

In [ ]:
def rutherford(th, Z1, Z2, en):
    en = en * 1e6 * e_chg
    return (Z1*Z2*e_chg**2 / (16*pi*eps_0*en))**2 / np.sin(np.pi/2 - th)**4 * 1e28

# Fit checker

In [ ]:
import os
from pytpc.fitting.mixins import constrain_angle, odrline

In [ ]:
class FitPlotter(object):
    def __init__(self, data_path, mcfitter):
        self.data_path = data_path
        self.fitter = mcfitter
        
    def plot_fit(self, fitres, run_num, evt_id):
        fp = os.path.join(self.data_path, 'clean_run_{:04d}.h5'.format(run_num))
        with h5py.File(fp, 'r') as hf:
            dataset = hf['/clean/{}'.format(int(evt_id))]
            raw_xyz = dataset[:]
            cx, cy = dataset.attrs['center'][:2]
            
        raw_xyz = pytpc.cleaning.apply_clean_cut(raw_xyz)
        xyz, (cu, cv) = self.fitter.preprocess(raw_xyz, center=[cx, cy], rotate_pads=False, last_tb=505)
#         xyz = pytpc.cleaning.nn_remove_noise(xyz)
        print('Num points:', len(xyz))
        
        tr = self.fitter.tracker.track_particle(fitres.x0, fitres.y0, fitres.z0, fitres.enu0, 
                                                fitres.azi0, fitres.pol0)
        
        arclen = np.sum(np.sqrt(np.sum(np.diff(tr[:, :3], axis=0)**2, -1)))
        print('Arclen:', arclen)
        print('Energy from arclen: ', self.fitter.gas.inverse_range(arclen, 1, 1))
        
        orig_guess = self.fitter.guess_parameters(fitres)
        orig_tr = self.fitter.tracker.track_particle(*orig_guess)
        
        xyz['cx'] = xyz.u - cu
        xyz['cy'] = xyz.v - cv
        xyz['cr'] = np.hypot(xyz.cx, xyz.cy)
        xyz['cth'] = np.unwrap(constrain_angle(np.arctan2(-cv, -cu) - np.arctan2(xyz.cy, xyz.cx)), discont=1.8 * pi)
        if np.abs(xyz['cth'].min() - 2 * pi) < 5 * degrees:
            xyz['cth'] -= 2 * pi
        
        fig, ax = plt.subplots(3, 2, figsize=(12, 14))
        fig.subplots_adjust(hspace=0.4, wspace=0.4)
        ax[0, 0].plot(xyz.w / 1000, xyz.u / 1000, '.')
        ax[0, 0].plot(tr[:, 2], tr[:, 0])
        ax[0, 0].plot(orig_tr[:, 2], orig_tr[:, 0], '--')
        ax[0, 0].set_xlabel('w [m]')
        ax[0, 0].set_ylabel('u [m]')
        
        ax[0, 1].plot(xyz.w / 1000, xyz.v / 1000, '.')
        ax[0, 1].plot(tr[:, 2], tr[:, 1])
        ax[0, 1].plot(orig_tr[:, 2], orig_tr[:, 1], '--')
        ax[0, 1].set_xlabel('w [m]')
        ax[0, 1].set_ylabel('v [m]')
        
        ax[1, 0].plot(xyz.w/1000, np.hypot(xyz.u, xyz.v) / 1000, '.')
        ax[1, 0].plot(tr[:, 2], np.hypot(tr[:, 0], tr[:, 1]))
        ax[1, 0].plot(orig_tr[:, 2], np.hypot(orig_tr[:, 0], orig_tr[:, 1]), '--')
        ax[1, 0].set_xlabel('w [m]')
        ax[1, 0].set_ylabel('$\sqrt{u^2 + v^2}$ [m]')
        
        ax[1, 1].plot(xyz.u / 1000, xyz.v / 1000, '.')
        ax[1, 1].plot(tr[:, 0], tr[:, 1])
        ths = np.linspace(0, 2*pi, 100)
        ax[1, 1].plot(fitres.rad_curv/1000 * np.cos(ths) + fitres.curv_ctr_x/1000, 
                      fitres.rad_curv/1000 * np.sin(ths) + fitres.curv_ctr_y/1000, 'r--')
        ax[1, 1].plot(cu/1000, cv/1000, 'k+', mew=1.5)
        ax[1, 1].plot(fitres.curv_ctr_x/1000, fitres.curv_ctr_y/1000, 'ro')
        ax[1, 1].set_aspect(1)
        ax[1, 1].set_xlabel('u [m]')
        ax[1, 1].set_ylabel('v [m]')
        
        slope = np.tan(fitres.lin_scat_ang - pi / 2)
        ydata = xyz.cr * xyz.cth
        ax[2, 0].plot(ydata, xyz.w, '.')
#         ax[2, 0].plot(ydata, odrline([fitres.lin_beam_int, slope], ydata))
        
        ax[2, 1].plot(xyz.w / 1000, xyz.a, '.')

# Results

In [ ]:
prodcut = res.run_num.isin(runtables[runtables.type == 'Production'].index)

In [ ]:
def find_kine_vert_en(mproj, mtarg, scat_ang, recoil_ke):
    E_recoil = recoil_ke + mtarg
    cm_ang = np.pi - 2*scat_ang
    p_sq = (mtarg * E_recoil - mtarg**2) / (1 - np.cos(cm_ang))
    sqrts = np.sqrt(mtarg**2 + p_sq) + np.sqrt(mproj**2 + p_sq)
    T = 1 / (2 * mtarg) * (sqrts**2 - (mtarg + mproj)**2)
    return T

In [ ]:
res['scat_ang'] = pi - res.pol0
res['cm_angle'] = pi - 2*res.scat_ang  # Approximation
res['totChi2'] = res['posChi2'] + res['enChi2']

In [ ]:
gas = pytpc.gases.InterpolatedGas(config['gas_name'], config['gas_pressure'])
if (gas.range(4.6*46 - 20., 46, 18) < 1.0):
    res['z0_corr'] = res.z0 + (1 - gas.range(4.6*46-20, 46, 18))
else:
    res['z0_corr'] = res.z0

In [ ]:
1 - gas.range(4.6*46 - 20., 46, 18)

In [ ]:
def find_vertex_energy(beam_intercept):
    ei = 4.6 * 46 - 20.
    ri = gas.range(ei, 46, 18)  # this is in meters
    rf = ri - (1000 - beam_intercept) / 1000
    ef = gas.inverse_range(rf, 46, 18)
    return ef
res['vert_en'] = find_vertex_energy(res.z0 * 1000) / 46
res['vert_en_corr'] = find_vertex_energy(res.z0_corr * 1000) / 46
res['kine_vert_en'] = find_kine_vert_en(46*p_mc2, 1*p_mc2, res.scat_ang, res.enu0) / 46

In [ ]:
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.cm import ScalarMappable

In [ ]:
ens = np.arange(1.0, 3.25, 0.25)
angs = np.linspace(50, 80, 1000) * degrees

with sns.husl_palette(len(ens)) as pal, sns.plotting_context('talk', font_scale=1.2):
    for en in ens:
        val = find_kine_vert_en(46*p_mc2, p_mc2, angs, en) / 46
        der = np.diff(val) / (np.diff(angs) / degrees)
        plt.plot(angs[1:] / degrees, der)
        
    cmap = ListedColormap(pal) 
    sm = ScalarMappable(norm=Normalize(ens.min(), ens.max()), cmap=cmap)
    sm._A = []
    plt.colorbar(sm, ticks=ens).set_label('Proton energy [MeV]')
    plt.xlabel('Scattering angle (lab) [deg]')
    plt.ylabel(r'$dE_\mathrm{Ar}/d\theta$ [MeV/(u-deg)]')
    
#     plt.savefig('/Users/josh/Desktop/Figures for committee/angdep.pdf')

# Chi^2 Behavior

In [ ]:
posChi2Max = 14
enChi2Max = 6
linChi2Max = 60

In [ ]:
plt.hist(res.posChi2.dropna().values, bins=100);#, range=(0, 20));
plt.vlines(posChi2Max, *plt.ylim())

In [ ]:
with sns.plotting_context('paper'), sns.axes_style(rc={'font.family': 'Arial'}):
    plt.figure(figsize=(3.5, 2.75))

    bins, edges = np.histogram(res.enChi2.values, bins=100, range=(0, 70));

    plt.step(edges[:-1], bins, color='k', label='Energy $\chi^2$', linewidth=1)
    plt.fill_between(edges[:-1], bins, step='pre', where=edges[:-1] < enChi2Max, facecolor='C0', edgecolor='none',
                     label='Proton tracks')
    plt.fill_between(edges[:-1], bins, step='pre', where=(edges[:-1] > 10) & (edges[:-1] < 36), 
                     edgecolor='none', facecolor='C3', label='Carbon tracks')

    plt.xlim(0, 70)
    plt.ylim(0)
    # plt.vlines(enChi2Max, *plt.ylim())
    # plt.axvspan(0, enChi2Max, zorder=-1, facecolor=)
    # plt.axvspan(12, 40, facecolor='none', hatch='//')
    sns.despine()
    plt.xlabel(r'$\chi^2_\mathrm{en}$')
    plt.ylabel('Count')

    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

    plt.legend(frameon=False)
    plt.tight_layout()

#     plt.savefig('/Users/josh/Documents/Papers/attpc-nim-paper/Figures/enchi2.pdf', bbox_inches='tight')

In [ ]:
plt.hist(res.totChi2.dropna().values, bins=100, range=(0, 200));
plt.vlines(posChi2Max + enChi2Max, *plt.ylim())

In [ ]:
plt.hist(res.lin_chi2, bins=100, range=(0, 1000));
plt.vlines(linChi2Max, *plt.ylim())

In [ ]:
plt.hist(res.vertChi2, bins=100, range=(0, 20));
# plt.vlines(, *plt.ylim())

In [ ]:
physicalcut = (res.scat_ang < pi/2)
goodfitcut = (res.posChi2 < posChi2Max) & (res.enChi2 < enChi2Max) # & (res.lin_chi2 < linChi2Max)
# goodfitcut = (res.totChi2 < 200) & (res.lin_chi2 < 200)

In [ ]:
goodidx = goodfitcut[goodfitcut].index

In [ ]:
print('goodfitcut: {} / {} = {}'.format(len(res[goodfitcut]), len(res), len(res[goodfitcut])/len(res)))

In [ ]:
lowruns = res.run_num < 166
hiruns = res.run_num >= 166

In [ ]:
from pytpc.fitting import MCFitter
import yaml

with open('../fitters/config_e15503b.yml', 'r') as f:
    config = yaml.load(f)
with open('../fitters/config_e15503b_macmini.yml', 'r') as f:
    patch = yaml.load(f)
config.update(patch)

# config['tilt'] = 6.0
fitter = MCFitter(config)

def find_mean_ampl_and_arclen(fitres):
    fp = '/Users/josh/Documents/Data/Merged/e15503b/clean/clean_run_{:04d}.h5'.format(int(fitres.run_num))
    with h5py.File(fp, 'r') as hf:
        dataset = hf['/clean/{}'.format(int(fitres.evt_id))]
        raw_xyz = dataset[:]
        
    raw_xyz = apply_clean_cut(raw_xyz)

    tr = fitter.tracker.track_particle(fitres.x0, fitres.y0, fitres.z0, fitres.enu0, 
                                            fitres.azi0, fitres.pol0)

    arclen = np.sum(np.sqrt(np.sum(np.diff(tr[:, :3], axis=0)**2, -1)))
    
    return raw_xyz[:, 3].max(), arclen

# Kinematics cuts

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & prodcut & physicalcut]
    plt.hist2d(np.cos(d.cm_angle), d.enu0 / d.vert_en_corr, bins=100, cmap='viridis', norm=LogNorm(),
               range=((-1, 1), (0, 5)))
    
    xs = np.linspace(0, pi/2, 200)
    p = find_proton_params(xs, 46*p_mc2, p_mc2, p_mc2, 46*p_mc2, 46.0)[1] - p_mc2
    plt.fill_between(np.cos(pi - 2*xs), p + 0.1, p - 0.1, facecolor=(1, 1, 1, 0.2), edgecolor=(1, 1, 1, 1))
    plt.xlabel('cos(lab scattering angle)')
    plt.ylabel('Proton energy (normalized to 1.0 MeV/u beam)')
    
    plt.colorbar().set_label('Count')

# Plots 

In [ ]:
xb, xe, fig = plt.hist(res[goodfitcut & physicalcut].z0 * 1000, bins=np.arange(850, 1100, 5));

from scipy.stats import norm
def fitfunc(x, mean, sigma, scale, offset):
    return scale * (1 - norm.cdf(x, loc=mean, scale=sigma)) + offset

fparam, ferr = scipy.optimize.curve_fit(fitfunc, xe[:-1], xb, p0=(1000, 5, 650, 0))
print(fparam)
print('FWHM = {:0.4f} mm'.format(fparam[1] * 2.355))

xs = np.linspace(xe.min(), xe.max(), 1000)
plt.plot(xs, fitfunc(xs, *fparam), 'k')

plt.xlim(xs.min(), xs.max())

plt.xlabel('Vertex position in z [m]')
plt.ylabel('Count')

In [ ]:
with sns.plotting_context('paper'), sns.axes_style(rc={'font.family': 'Arial'}):
    fig, ax_main = plt.subplots(figsize=(3.5, 2.6))
    
    binsize = 5
    
    ax_main.hist(res[goodfitcut & physicalcut].z0 * 1000, bins=np.arange(0, 1700, binsize), histtype='stepfilled');

    ax_main.set_xlabel('Vertex position in z [mm]')
    ax_main.set_ylabel('Count')

    ax_sub = fig.add_axes([0.6, 0.35, 0.3, 0.4])
    
    xb, xe = np.histogram(res[goodfitcut & physicalcut].z0 * 1000, bins=np.arange(900, 1050, binsize))
    
    from scipy.stats import norm
    def fitfunc(x, mean, sigma, scale, offset):
        return scale * (1 - norm.cdf(x, loc=mean, scale=sigma)) + offset

    fparam, ferr = scipy.optimize.curve_fit(fitfunc, xe[:-1], xb, p0=(1000, 5, 650, 0))
    print(fparam)
    print('FWHM = {:0.4f} mm'.format(fparam[1] * 2.355))
    
    xs = np.linspace(xe.min(), xe.max(), 1000)
    ax_sub.errorbar(xe[:-1], xb, yerr=np.sqrt(xb), fmt='.', elinewidth=1, capthick=1, capsize=1)
    ax_sub.plot(xs, fitfunc(xs, *fparam), 'k')
    ax_sub.set_xlim(xe.min(), xe.max())
    ax_sub.set_xticks(range(940, 1050, 40))
    ax_sub.set_yticks([])
    
#     ax_sub.set_xlabel('Vertex position in z [mm]')
#     ax_sub.set_ylabel('Count')
    
#     ax_main.set_ylim(0, 1020)
    
    sns.despine()

    # plt.tight_layout()
#     plt.savefig('/Users/josh/Documents/Papers/attpc-nim-paper/Figures/z_dist.pdf', bbox_inches='tight')

In [ ]:
with sns.plotting_context('paper'), sns.axes_style('ticks'):
    fig, ax_main = plt.subplots(figsize=(3.5, 2.6))
    
    binsize = 10e-3
    
    ax_main.hist(res[goodfitcut & physicalcut].vert_en, bins=np.arange(0, 6, binsize), histtype='stepfilled');

    ax_main.set_xlabel('Vertex position in z [mm]')
    ax_main.set_ylabel('Count')

    ax_sub = fig.add_axes([0.6, 0.35, 0.3, 0.4])
    
    xb, xe = np.histogram(res[goodfitcut & physicalcut].vert_en, bins=np.arange(3.8, 4.5, binsize))
    
    from scipy.stats import norm
    def fitfunc(x, mean, sigma, scale, offset):
        return scale * (1 - norm.cdf(x, loc=mean, scale=sigma)) + offset

    fparam, ferr = scipy.optimize.curve_fit(fitfunc, xe[:-1], xb, p0=(4.2, 40e-3, 400, 0))
    print(fparam)
    print('FWHM = {:0.4f} keV'.format(fparam[1] * 2.355 * 1000))
    
    xs = np.linspace(xe.min(), xe.max(), 1000)
    ax_sub.errorbar(xe[:-1], xb, yerr=np.sqrt(xb), fmt='.', elinewidth=1, capthick=1, capsize=1)
    ax_sub.plot(xs, fitfunc(xs, *fparam), 'k')
    ax_sub.set_xlim(xe.min(), xe.max())
#     ax_sub.set_xticks(range(940, 1050, 40))
    ax_sub.set_yticks([])
    
#     ax_sub.set_xlabel('Vertex position in z [mm]')
#     ax_sub.set_ylabel('Count')
    
#     ax_main.set_ylim(0, 1020)
    
    sns.despine()

    # plt.tight_layout()
#     plt.savefig('/Users/josh/Documents/Pape/rs/attpc-nim-paper/Figures/z_dist.pdf', bbox_inches='tight')

In [ ]:
plt.hist(res[goodfitcut & physicalcut].enu0, bins=np.arange(0, 10.1, 0.1), log=True);
plt.xlabel('Proton initial energy (lab) [MeV]')
plt.ylabel('Count')
# plt.ylim(0.1, 1e4)

In [ ]:
plt.hist(res[goodfitcut & physicalcut & prodcut].cm_angle / degrees, bins=250);
plt.xticks([25, 50, 75, 100, 125])
plt.xlabel('CM angle [deg]')
plt.ylabel('Count')

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & physicalcut]
    plt.hist2d(d.run_num, d.enu0, bins=(np.arange(d.run_num.min(), d.run_num.max(), 1), 
                                        np.arange(0, 6.1, 0.1)),
               cmap='viridis');
    plt.vlines(217, *plt.ylim(), colors='white')
#     plt.xlabel('Proton initial energy (lab) [MeV]')
#     plt.ylabel('Count')
    # plt.ylim(0.1, 1e4)

In [ ]:
gsrescut = ((res.cm_angle > 70 * degrees) & (res.cm_angle < 120 * degrees) 
            & (res.vert_en < 2.2) & (res.vert_en > 1.5))

In [ ]:
with sns.plotting_context('talk', font_scale=1.0):
    with sns.axes_style('white'):
        d = res[goodfitcut & (res.run_num > 0)]
        plt.hist2d(d.cm_angle/degrees, d.enu0, range=((10, 140), (0, 6)), bins=100, cmap='viridis', norm=LogNorm());
        plt.xlabel('CM scattering angle [deg]')
        plt.ylabel('Reconstructed 46Ar vertex energy [MeV/u]')
        plt.colorbar().set_label('Count')
    #     xs = np.linspace(0, 5, 10)
    #     plt.plot(xs, xs, 'w-', linewidth=1)

        ths = np.linspace(0, pi/2, 180)
        p_ens = pytpc.relativity.find_proton_params(ths, 46*p_mc2, p_mc2, p_mc2, 46*p_mc2, 46*4.6)[1] - p_mc2
        cm_angs = pi - 2*ths
    #     plt.plot(cm_angs/degrees, p_ens, 'k-', linewidth=0.5, alpha=1)

#         plt.savefig('/Users/josh/Desktop/Figures/data_2d_spectrum.pdf', transparent=True)

In [ ]:
d = res[goodfitcut & prodcut & physicalcut]
plt.hist(d.rad_curv.dropna(), bins=100);

In [ ]:
d = res[goodfitcut & prodcut & physicalcut].dropna()
plt.hist2d(d.brho, d.kine_vert_en, bins=100,
           cmap='viridis', norm=LogNorm());

In [ ]:
with sns.plotting_context('paper'), sns.axes_style(rc={'font.family': 'Arial'}):
    plt.figure(figsize=(3.5, 3.5*2/3))
    res['vertdiff'] = res.kine_vert_en - res.vert_en
    d = res[goodfitcut & physicalcut & prodcut]
    plt.hist(d.vertdiff * 1000, bins=np.arange(-2000, 2000, 25), histtype='stepfilled');

    plt.xlabel(r'$E_{\mathrm{kin}} - E_{\mathrm{pos}}$ [keV/u]')
    plt.ylabel('Count')

    sns.despine()
#     plt.tight_layout()

#     plt.savefig('/Users/josh/Documents/Papers/attpc-nim-paper/Figures/exp_endiff.pdf', bbox_inches='tight')

In [ ]:
res['vertdiff'] = res.kine_vert_en - res.vert_en
d = res[goodfitcut & physicalcut & ~gsrescut]
dgsres = res[goodfitcut & physicalcut & gsrescut]
plt.hist([d.vertdiff, dgsres.vertdiff], bins=100, log=False, range=(-5, 5), stacked=True, rwidth=1);
vertdiffmed = -1 #d.vertdiff.median()
sigma = 0.200

vertencut = np.abs(res.vertdiff - vertdiffmed) < sigma

plt.vlines((vertdiffmed, vertdiffmed - sigma, vertdiffmed + sigma), *plt.ylim())
# plt.ylim(0.1, 1e4)
plt.xlabel('(Kine. vertex en.) – (pos. vertex en.) [MeV/u]')
plt.ylabel('Count')

# plt.savefig('/Users/josh/Desktop/Plots for Daniel/exp_endiff.pdf')

In [ ]:
from scipy.stats import norm
from scipy.optimize import curve_fit

def gaus(x, s, a):
    return a * poisson.pdf(x, loc=0, scale=s)

def lorentz(x, x0, gamma, a):
    return a / ((x - x0)**2 + 0.25 * gamma**2)

In [ ]:
with sns.plotting_context('talk', font_scale=1.2):
    res['vertdiff'] = res.kine_vert_en - res.vert_en
    d = res[goodfitcut & physicalcut
#             & (res.vert_en > 0.75) & (res.vert_en < 1.2)
#             & (res.cm_angle > 40*degrees) & (res.cm_angle < 50*degrees)
           ]
    cts, edges, fig = plt.hist(d.vertdiff, bins=50, log=False, range=(-0, 5))
    
    fparam, fres = curve_fit(lorentz, edges[:-1], cts)
    print(fparam)
    
    xs = np.linspace(-5, 5, 1000)
    plt.plot(xs, lorentz(xs, *fparam))
    
    plt.xlabel('(Kine. vertex en.) – (pos. vertex en.) [MeV/u]')
    plt.ylabel('Count')
    
#     plt.ylim(0, 1000)

#     plt.savefig('/Users/josh/Desktop/Figures for committee/exp_verten_diff.pdf')

In [ ]:
with sns.plotting_context('talk', font_scale=1.2):
    res['vertdiff'] = res.kine_vert_en - res.vert_en
    d = res[goodfitcut & physicalcut
#             & (res.vert_en > 0.75) & (res.vert_en < 1.2)
#             & (res.cm_angle > 40*degrees) & (res.cm_angle < 50*degrees)
           ]
    cts, edge = np.histogram(d.vertdiff, bins=100, range=(-5, 5))
    
    plt.step(edge[:-1], cts - lorentz(edge[:-1], *fparam), where='post')
#     plt.step(edge[:-1], cts, where='post')
    
    plt.ylim(-200, 100)

#     plt.savefig('/Users/josh/Desktop/Figures for committee/exp_verten_diff.pdf')

In [ ]:
d = res[goodfitcut & physicalcut & vertencut]
plt.hist(d.vertdiff, bins=100, log=False, range=(-5, 5));

plt.vlines((vertdiffmed, vertdiffmed - sigma, vertdiffmed + sigma), *plt.ylim())
# plt.ylim(0.1, 1e4)
plt.xlabel('(Kine. vertex en.) – (pos. vertex en.) [MeV/u]')
plt.ylabel('Count')

In [ ]:
print("Vert en cut keeps {} / {}".format(len(res[goodfitcut & physicalcut & vertencut]), 
                                         len(res[goodfitcut & physicalcut])))

In [ ]:
with sns.plotting_context('talk'), sns.axes_style('white'):
    e = 0.5
    d = res[goodfitcut & physicalcut]
    fig, ax = plt.subplots(1, 2, squeeze=True, figsize=(11, 4))
    *junk, c0 = ax[0].hist2d(d.cm_angle/degrees, d.vert_en, range=((10, 140), (0, 6)), bins=100, cmap='viridis', norm=LogNorm());
    *junk, c1 = ax[1].hist2d(d.cm_angle/degrees, d.kine_vert_en, range=((10, 140), (0, 6)), bins=100, cmap='viridis', norm=LogNorm());
    plt.colorbar(c0, ax=ax[0]).set_label('Count')
    plt.colorbar(c1, ax=ax[1]).set_label('Count')
    ax[0].set_xlabel('CM scattering angle [deg]')
    ax[1].set_xlabel('CM scattering angle [deg]')
    ax[0].set_ylabel('46Ar vertex energy [MeV/u]')
    
    ax[0].set_title('From vertex position')
    ax[1].set_title('From kinematics')
    

#     plt.savefig('/Users/josh/Desktop/Figures for committee/exp_pos_2d.pdf', transparent=True, bbox_inches='tight')

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & physicalcut & prodcut & (res.vme_group == 0)]
    plt.hist2d(d.ic_height, d.vertdiff, bins=100, range=((4500, 6500), (-10, 10)), cmap='viridis', norm=LogNorm());
    plt.xlabel('IC signal amplitude')
    plt.ylabel('46Ar Vertex energy difference [MeV/u]')
    plt.colorbar().set_label('Count')

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & physicalcut & prodcut & (res.vme_group == 0)]
    plt.hist2d(d.ic_cfd_pos, d.vertdiff, bins=100, range=((147, 155), (-6, 6)), cmap='viridis', norm=LogNorm());
    plt.xlabel('IC signal timing [time buckets]')
    plt.ylabel('46Ar Vertex energy difference [MeV/u]')
    plt.colorbar().set_label('Count')

In [ ]:
plt.hist(res[goodfitcut & physicalcut].kine_vert_en, 
         bins=np.arange(0, 8.1, 0.1));
plt.xlabel('46Ar lab vertex energy from kinematics [MeV/u]')
plt.ylabel('Count')

In [ ]:
plt.hist(res[goodfitcut & physicalcut & prodcut].vert_en, bins=np.arange(0, 4.8, 0.01));
plt.xlabel('46Ar energy at reaction vertex [MeV/u]')
plt.ylabel('Count')
sns.despine()
# plt.savefig('/Users/josh/Desktop/Figures/energy_spectrum.pdf', transparent=True, bbox_inches='tight')

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & physicalcut].copy()
#     d.x0 += -0.01 - 0.0039
#     d.y0 += -0.104 + 0.105
    jp = sns.jointplot(d.x0 * 1000, d.y0 * 1000, kind='hex')
    jp.set_axis_labels('Vertex x position [mm]', 'Vertex y position [mm]')
    jp.ax_joint.plot(0, 0, 'r+', ms=20, mew=2)
    mean_pt = (d.x0.median() * 1000, d.y0.median() * 1000)
    print("Center at", mean_pt)
    jp.ax_joint.plot(*mean_pt, 'gx', ms=20, mew=2)

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut]
    plt.hist2d(d.vert_en, d.kine_vert_en, range=((0, 5), (0, 6)), bins=100, cmap='viridis', norm=LogNorm());
    plt.xlabel('Vertex energy from vertex position [MeV/u]')
    plt.ylabel('Vertex energy from kinematics [MeV/u]')
    plt.colorbar().set_label('Count')
    xs = np.linspace(0, 5, 10)
    plt.plot(xs, xs, 'w-', linewidth=1)

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & prodcut & physicalcut]
    plt.hist2d(d.cm_angle / degrees, d.vertdiff, range=((0, 180), (-4, 4)), bins=100, cmap='viridis', norm=LogNorm());
    plt.xlabel('CM scattering angle [deg]')
    plt.ylabel('Kine vert en - pos vert en [MeV/u]')
    plt.colorbar().set_label('Count')

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & prodcut & physicalcut]
    plt.hist2d(d.z0, d.vertdiff, range=((0, 1.1), (-4, 4)), bins=100, cmap='viridis');
    plt.xlabel('Vertex Z position [m]')
    plt.ylabel('Kine vert en - pos vert en [MeV/u]')
    plt.colorbar().set_label('Count')

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & prodcut & physicalcut]
    plt.hist2d(np.hypot(d.x0, d.y0), d.vertdiff, range=((-0.05, 0.05), (-4, 4)), bins=100, cmap='viridis');
    plt.xlabel('Vertex Z position [m]')
    plt.ylabel('Kine vert en - pos vert en [MeV/u]')
    plt.colorbar().set_label('Count')

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & prodcut & physicalcut]
    plt.hist2d(d.enu0, d.vertdiff, range=((0, 5), (-4, 4)), bins=100, cmap='viridis');
    plt.xlabel('Proton energy [MeV]')
    plt.ylabel('Kine vert en - pos vert en [MeV/u]')
    plt.colorbar().set_label('Count')

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & prodcut & physicalcut]
    plt.hist2d(d.vertChi2, d.vertdiff, range=((0, 2), (-4, 4)), bins=100, cmap='viridis');
    plt.xlabel('Vertex position chi2')
    plt.ylabel('Kine vert en - pos vert en [MeV/u]')
    plt.colorbar().set_label('Count')

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & prodcut & physicalcut]
    
    fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(12, 6))
    plt.sca(ax[0])
    plt.hist2d(d.vert_en_corr, d.vertdiff, range=((0, 8), (-4, 4)), bins=100, cmap='viridis');
    plt.xlabel('Pos vert en [MeV/u]')
    plt.ylabel('Kine vert en - pos vert en [MeV/u]')
    plt.colorbar(orientation='horizontal').set_label('Count')
    
    plt.sca(ax[1])
    plt.hist2d(d.kine_vert_en, d.vertdiff, range=((0, 8), (-4, 4)), bins=100, cmap='viridis');
    plt.xlabel('Kine vert en [MeV/u]')
    plt.colorbar(orientation='horizontal').set_label('Count')
    
    plt.tight_layout()

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & prodcut & physicalcut]
    
    fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(12, 6))
    plt.sca(ax[0])
    plt.hist2d(d.vert_en, d.z0, range=((0, 8), (0, 1.1)), bins=100, cmap='viridis');
    plt.xlabel('Pos vert en [MeV/u]')
    plt.ylabel('Vertex Z position [m]')
    plt.colorbar(orientation='horizontal').set_label('Count')
    
    plt.sca(ax[1])
    plt.hist2d(d.kine_vert_en, d.z0, range=((0, 8), (0, 1.1)), bins=100, cmap='viridis');
    plt.xlabel('Kine vert en [MeV/u]')
    plt.colorbar(orientation='horizontal').set_label('Count')
    
    plt.tight_layout()

In [ ]:
with sns.axes_style('white'):
    dtheta = 1.0
    dE = 50e-3
    
    min_vert_en = 2.0
    delta_vert_en = 0.05
    
    histbins = (np.arange(0, 100 + dtheta, dtheta),
                np.arange(0, 6 + dE, dE))
    d = res[goodfitcut & gsrescut
#             & (res.vert_en_corr > min_vert_en) & (res.vert_en_corr < min_vert_en + delta_vert_en)
           ]
    plt.hist2d(d.scat_ang / degrees, d.enu0, cmap=cmaps.viridis, bins=histbins, norm=LogNorm());
    plt.colorbar().set_label('Count')
    plt.xlabel('CM Angle [deg]')
    plt.ylabel('Proton energy (lab) [MeV]')
    
    ths = np.linspace(0, pi/2, 180)
    p_ens = pytpc.relativity.find_proton_params(ths, 46*p_mc2, p_mc2, p_mc2, 46*p_mc2, 46*min_vert_en)[1] - p_mc2
    cm_angs = pi - 2*ths
    plt.plot(ths/degrees, p_ens, 'k-', linewidth=0.5, alpha=1)

---
# Binning

In [ ]:
dtheta = 5.0
dE = 50 * 1e-3
thbins = np.arange(0, 130 + dtheta, dtheta) * degrees
enbins = np.arange(0, 5 + dE, dE)

def bin_data(data, thbins, enbins):
    b, x, y = np.histogram2d(data['vert_en'], data['cm_angle'], bins=(enbins, thbins))
    return pd.DataFrame(b, index=x[:-1], columns=y[:-1])

def show_prebinned_hist2d(data, **kwargs):
    plt.imshow(data.values, cmap='viridis', aspect='auto', interpolation='none', origin='lower',
               extent=(data.columns.min() / degrees, data.columns.max() / degrees, 
                       data.index.min(), data.index.max()), **kwargs)
    plt.xlabel('CM angle [deg]')
    plt.ylabel('${}^{46}\mathrm{Ar}$ vertex energy [MeV/u]')
    plt.colorbar().set_label('Count')

In [ ]:
binned = bin_data(res[goodfitcut & prodcut & physicalcut], thbins, enbins)

In [ ]:
mcres_h5 = os.path.join(os.path.dirname(os.path.dirname(rawdbs[0])), 'mcresults.h5')
print(mcres_h5)

In [ ]:
binned.to_hdf(mcres_h5, 'binned_res')

---

In [ ]:
def histerr(x, bins, **kwargs):
    xb, xe = np.histogram(x, bins=bins)
    plt.errorbar(xe[:-1] + (xe[1] - xe[0]) / 2, xb, yerr=np.sqrt(xb), fmt='b.', ecolor='b', elinewidth=1,
                 capthick=1, capsize=1, **kwargs)
    
dtheta = 5.0
dE = 50 * 1e-3
histbins = (np.arange(0, 180 + dtheta, dtheta),
            np.arange(0, 6 + dE, dE))
    
d = res[goodfitcut & vertencut].copy()
d['thbin'] = histbins[0][np.digitize(d.cm_angle / degrees, histbins[0])]
fg = sns.FacetGrid(d, col='thbin', col_order=range(35, 80, 5), col_wrap=3, sharey=False)
fg.map(histerr, 'enu0', bins=histbins[1], markersize=5)
# fg.map(plt.hist, 'enu0', bins=histbins[1], histtype='stepfilled')
# fg.set_titles(r'$\theta_{{CM}} = {col_name}^{{\circ}}$')
fg.set_axis_labels('Proton energy [MeV]', 'Count')

In [ ]:
sns.set_context('talk')

In [ ]:
def histerr(x, bins, **kwargs):
    xb, xe = np.histogram(x, bins=bins)
    plt.errorbar(xe[:-1] + (xe[1] - xe[0]) / 2, xb, yerr=np.sqrt(xb), fmt='.', elinewidth=1, **kwargs)
#     plt.step(xe[:-1], xb, linewidth=1, where='post')

dtheta = 5.0
dE = 50 * 1e-3
histbins = (np.arange(0, 180 + dtheta, dtheta),
            np.arange(0, 5.0 + dE, dE))

d = res[goodfitcut & physicalcut & prodcut #& vertencut
       ].copy()
d['thbin'] = histbins[0][np.digitize(d.cm_angle / degrees, histbins[0])]
fg = sns.FacetGrid(d, col='thbin', col_order=range(40, 100, 5), col_wrap=4, sharey=False, size=4)
fg.map(histerr, 'vert_en', bins=histbins[1], markersize=5)
fg.set_axis_labels('${}^{46}\mathrm{Ar}$ vertex energy [MeV/u]', 'Count')
fg.set_titles(r'$\theta_\mathrm{{CM}} = {col_name}^\circ$')
plt.tight_layout()
for ax in fg.axes.ravel():
    ax.set_xticks(np.arange(6))
# plt.savefig('/Users/josh/Desktop/Plots for Daniel/exp_pos_all_spectra.pdf')
del fg

In [ ]:
def histerr(x, bins, **kwargs):
    xb, xe = np.histogram(x, bins=bins)
    plt.errorbar(xe[:-1] + (xe[1] - xe[0]) / 2, xb, yerr=np.sqrt(xb), fmt='b.', ecolor='b', elinewidth=1,
                 capthick=1, capsize=1, **kwargs)
#     plt.step(xe[:-1], xb, linewidth=1, where='post')
    
dtheta = 5.0
dE = 50 * 1e-3
histbins = (np.arange(0, 180 + dtheta, dtheta),
            np.arange(0, 5.0 + dE, dE))
chosen_theta = 65
    
d = res[goodfitcut & physicalcut & prodcut].copy()
d['thbin'] = histbins[0][np.digitize(d.cm_angle / degrees, histbins[0])]
subplotdata = d[d.thbin == chosen_theta]

histerr(subplotdata.vert_en, histbins[1])
# plt.hist(subplotdata.kine_vert_en, bins=histbins[1], histtype='stepfilled')

plt.xlabel('46Ar vertex energy [MeV/u]')
plt.ylabel('Count')
plt.title(r'$\theta_{{CM}} = {}^\circ$'.format(chosen_theta))
plt.tight_layout()
# plt.savefig('/Users/josh/Desktop/Plots for Daniel/exp_pos_spectrum_65.pdf')

In [ ]:
len(res[goodfitcut & prodcut & physicalcut])

In [ ]:
def histerr(x, bins, **kwargs):
    xb, xe = np.histogram(x, bins=bins)
    plt.errorbar(xe[:-1] + (xe[1] - xe[0]) / 2, xb, yerr=np.sqrt(xb), fmt='b.', ecolor='b', elinewidth=1,
                 capthick=1, capsize=1, **kwargs)
#     plt.step(xe[:-1], xb, linewidth=1, where='post')
    
dtheta = 5
dE = 50 * 1e-3
histbins = (np.arange(0, 180 + dtheta, dtheta),
            np.arange(3.0, 4.6 + dE, dE))
    
d = res[goodfitcut & physicalcut & prodcut & vertencut].copy()
d['thbin'] = histbins[0][np.digitize(d.cm_angle / degrees, histbins[0])]
fg = sns.FacetGrid(d, col='thbin', col_order=range(35, 80, dtheta), col_wrap=3, sharey=False)
fg.map(histerr, 'vert_en_corr', bins=histbins[1], markersize=5)
# fg.map(plt.hist, 'kine_vert_en', bins=histbins[1], histtype='stepfilled')
# fg.set_titles(r'$\theta_{{CM}} = {col_name}^{{\circ}}$')
fg.set_axis_labels('46Ar vertex energy [MeV/u]', 'Count')

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & physicalcut & prodcut & vertencut]
    plt.hist2d(d.cm_angle/degrees, d.kine_vert_en, bins=(np.arange(0, 185, 5), np.arange(0, 5.0, 0.05)), 
               cmap='viridis');
    plt.colorbar()
    plt.xlabel('CM scattering angle [deg]')
    plt.ylabel('Kine. vert. en. [MeV/u]')

In [ ]:
len(res[goodfitcut & vertencut & physicalcut])

In [ ]:
def histerr(x, bins, **kwargs):
    xb, xe = np.histogram(x, bins=bins)
    plt.errorbar(xe[:-1] + (xe[1] - xe[0]) / 2, xb, yerr=np.sqrt(xb), fmt='b.', ecolor='b', elinewidth=1,
                 capthick=1, capsize=1, **kwargs)
    
dtheta = 1.0 * degrees
dE = 100e-3
histbins = (np.arange(35*degrees, 90*degrees + dtheta, dtheta),
            np.arange(0, 5 + dE, dE))
    
d = res[goodfitcut].copy()
d['enbin'] = (np.digitize(d.vert_en, histbins[1]))
d.loc[d.enbin == d.enbin.max(), 'enbin'] = 0
d.loc[:, 'enbin'] = histbins[1][d.enbin.values]
fg = sns.FacetGrid(d, col='enbin', col_order=np.round(np.arange(0, 5, 0.5), 1), col_wrap=3, sharey=True)
# fg.map(histerr, 'scat_ang', bins=histbins[0], markersize=5)
fg.map(plt.hexbin, 'scat_ang', 'enu0')
# fg.map(plt.hist, 'enu0', bins=histbins[1], histtype='stepfilled')
# fg.set_titles(r'$\theta_{{CM}} = {col_name}^{{\circ}}$')
# fg.set_axis_labels('Proton energy [MeV]', 'Count')

# Exporting

In [ ]:
# res[goodfitcut & prodcut & physicalcut].to_hdf(
#     '/Users/josh/Documents/Data/Results/e15503b/mcmin/Sep23/goodevts.h5',
#     'goodevts'
# )

# Cross sections

## Import deteff

In [ ]:
dE = 0.050  # MeV
dTheta = 1.0 * degrees
enbins = np.arange(0, 6 + dE, dE)
thbins = np.arange(0, pi + dTheta, dTheta)

enxs = (enbins[:-1] + enbins[1:]) / 2
thxs = (thbins[:-1] + thbins[1:]) / 2

d = res[goodfitcut]
binned_res, *junk = np.histogram2d(d.cm_angle, d.enu0, bins=(thbins, enbins))

In [ ]:
deteff_res = pd.read_hdf('/Users/josh/Documents/Data/Results/e15503b/deteff/Feb5/results.h5', 'res')

In [ ]:
deteff_good = deteff_res[deteff_res.good]
binned_deteff_good, *junk = np.histogram2d(deteff_good.cm_angle, deteff_good.enu, bins=(thbins, enbins))
binned_deteff_all, *junk = np.histogram2d(deteff_res.cm_angle, deteff_res.enu, bins=(thbins, enbins))

binned_deteff = binned_deteff_good / binned_deteff_all
binned_deteff[np.where(binned_deteff_all == 0)] = 0

del deteff_good, binned_deteff_all, binned_deteff_good

In [ ]:
plt.imshow(binned_deteff.T, interpolation='nearest', origin='lower', cmap=cmaps.viridis)

In [ ]:
ruthxsec = np.array([[rutherford(th*degrees, 18, 1, en) for en in enxs] for th in thxs])

In [ ]:
def get_slice(lb, ub, xs):
    return np.where((xs >= lb) & (xs < ub))[0]

In [ ]:
def slice_data(data, enxs, thxs, thmin=None, thmax=None, enmin=None, enmax=None):
    if thmin is not None and thmax is not None:
        xs = enxs
        ys = data[get_slice(thmin, thmax, thxs)].sum(0)
    elif enmin is not None and enmax is not None:
        xs = thxs / degrees
        ys = data[:, get_slice(enmin, enmax, enxs)].sum(1)
    else:
        raise ValueError('Need th or en bounds')
    return xs, ys

In [ ]:
thstep = 2
minths = range(40, 70, thstep)
ncols = 3
nrows = ceil(len(minths) / ncols)
figsize = (ncols * 3, nrows * 3)
fig, ax = plt.subplots(nrows, ncols, sharex=True, sharey=False, figsize=figsize)
flatax = ax.ravel()
for i, thmin in enumerate(minths):
    plt.sca(flatax[i])
    
    xs, ys = slice_data(binned_res, enxs, thxs, thmin=thmin*degrees, thmax=(thmin+thstep)*degrees)
    plt.errorbar(xs, ys, yerr=np.sqrt(ys), fmt='.', elinewidth=1, capthick=1, capsize=1, zorder=100)
    
    deteff_xs, deteff_ys = slice_data(binned_deteff, enxs, thxs, thmin=thmin*degrees, thmax=(thmin+thstep)*degrees)
    plt.fill_between(deteff_xs, deteff_ys / thstep * plt.ylim()[1], step='post', alpha=0.3, zorder=1)
    
    plt.title(r'$\theta_{{CM}} \in [{}^\circ, {}^\circ)$'.format(thmin, thmin+thstep))
    if i % ncols == 0:
        plt.ylabel('Count')
    if i // nrows == nrows - 1:
        plt.xlabel('Proton Energy [MeV]')
plt.tight_layout()
for j in range(i+1, nrows*ncols):
    flatax[j].set_visible(False)

In [ ]:
thstep = 2
minths = range(40, 70, thstep)
ncols = 3
nrows = ceil(len(minths) / ncols)
figsize = (ncols * 3, nrows * 3)
fig, ax = plt.subplots(nrows, ncols, sharex=True, sharey=True, figsize=figsize)
flatax = ax.ravel()
for i, thmin in enumerate(minths):
    plt.sca(flatax[i])
    xs, ys = slice_data(binned_deteff, enxs, thxs, thmin=thmin*degrees, thmax=(thmin+thstep)*degrees)
    plt.fill_between(xs, ys / thstep, step='post')
    plt.title(r'$\theta_{{CM}} \in [{}^\circ, {}^\circ)$'.format(thmin, thmin+thstep))
    if i % ncols == 0:
        plt.ylabel('Count')
    if i // nrows == nrows - 1:
        plt.xlabel('Proton Energy [MeV]')
plt.tight_layout()
for j in range(i+1, nrows*ncols):
    flatax[j].set_visible(False)

# Fit plotting

In [ ]:
endiffs = []
extraps = []
percents = []

In [ ]:
from pytpc.fitting import MCFitter
import yaml

with open('../fitters/config_e15503b.yml', 'r') as f:
    config = yaml.load(f)
with open('../fitters/config_e15503b_macmini.yml', 'r') as f:
    patch = yaml.load(f)
config.update(patch)

# config['tilt'] = 6.0
fitter = MCFitter(config)
# fitter.sigma[-1] = 0
# fitter.sigma[0] = 0.005
# fitter.sigma[1] = 0.005
# fitter.sigma[2] = 0
# fitter.bfield_mag = 1.8

In [ ]:
fitpl = FitPlotter('/Users/josh/Documents/Data/Merged/e15503b/clean', fitter)

In [ ]:
i = 0

In [ ]:
i -= 1

In [ ]:
matching_en_cut = np.abs(res.vert_en - res.kine_vert_en) < 0.1

In [ ]:
i += 1
# i = np.random.randint(0, len(res[goodfitcut]))
# idx = res[goodfitcut].index[i] #.sort_values('enChi2', ascending=False).index[i]
# idx = res[goodfitcut & physicalcut & vertencut].sort_values('enu0', ascending=False).index[i]
e = 4.0
de = 0.05
idx = res[goodfitcut & physicalcut & prodcut
          & (res.enu0 < 1.2)
#           & (res.run_num == 115)
#           & (res.evt_id == 563)
#           & res.run_num.isin([76, 85, 99, 193, 263])
#           & (res.vert_en > 0) & (res.vert_en < 1.5)
#           & (res.cm_angle > 40*degrees) & (res.cm_angle < 45*degrees)
#           & (res.run_num > 217)
#           & (res.posChi2 < posChi2Max)
#           & (res.enChi2 < enChi2Max)
#         & (res.enChi2 > 15) & (res.enChi2 < 30) 
].sample(1).index[0]

# idx = res[goodfitcut & vertencut].sample(i).index[0]
# idx = res[goodfitcut & matching_en_cut].sample(i).index[0]
# idx = res.index[i]
# idx = res[(res.run_num == 85) & (res.evt_id == 80)].index[0]
fr = res.loc[idx]

########## RE-DO FIT HERE
# with h5py.File('/Users/josh/Documents/Data/Merged/e15503b/peaks/run_0075_peaks_bkd_subtract.h5', 'r') as f:
#     raw_xyz = f['/reduced_get_events/{}'.format(int(fr.evt_id))][:]
# fr = pd.Series(fitter.process_event(cfr, raw_xyz))
# fr['run_num'] = cfr.run_num
# fr['evt_id'] = cfr.evt_id
##########

print('Run {} event {}'.format(int(fr.run_num), int(fr.evt_id)))
print('linfit chi2:', fr.lin_chi2)
print('mcmin poschi2:   ', fr.posChi2)
print('mcmin  enchi2:   ', fr.enChi2)
print('kine vert_en:    ', fr.kine_vert_en)
print('z0 vert_en:      ', fr.vert_en)
print('z0 vert_en_corr: ', fr.vert_en_corr)
# print('fitter totChi2:', fr.totChi2)
# print('fitter success:', fr.success)
print('ctr:', fr[['x0', 'y0', 'z0', 'enu0', 'azi0', 'pol0']].values)
print('CM Angle: {:0.2f}   Lab angle: {:0.2f}'.format(fr.cm_angle / degrees, fr.scat_ang / degrees))
# print('Chi score: {:.0f}'.format(fr.score))
fitpl.plot_fit(fr, int(fr.run_num), fr.evt_id)
e1 = find_proton_params(fr.scat_ang, 46*p_mc2, p_mc2, p_mc2, 46*p_mc2, fr.vert_en * 46)[1] - p_mc2
e2 = find_proton_params(fr.scat_ang, 46*p_mc2, p_mc2, p_mc2, 46*p_mc2, fr.kine_vert_en * 46)[1] - p_mc2
# print('Energy from vert_en: ', e1)
# print('Range from that energy: ', gas.range(e1, 1, 1))
# print('Energy from kine_vert_en: ', e2)
# print('Range from that energy: ', gas.range(e2, 1, 1))

print('Mean amplitude:', find_mean_ampl_and_arclen(fr)[0])

In [ ]:
len(res[(res.posChi2 > posChi2Max) & (res.enChi2 < enChi2Max)])

In [ ]:
show_prebinned_hist2d(binned, norm=LogNorm())
plt.scatter(fr.cm_angle / degrees, fr.vert_en, c='C1', marker='x', s=200)

In [ ]:
frd = fr.to_dict()
del frd['brho']
frd

In [ ]:
with h5py.File('/Users/josh/Documents/Data/Merged/e15503b/clean/clean_run_{:04d}.h5'.format(int(fr.run_num)), 'r') as f:
    raw_xyz = f['/clean/{}'.format(int(fr.evt_id))][:]
raw_xyz = apply_clean_cut(raw_xyz)
xyz = fitter.preprocess(raw_xyz)

In [ ]:
res_track = fitter.tracker.track_particle(fr.x0, fr.y0, fr.z0, fr.enu0, fr.azi0, fr.pol0)

In [ ]:
exp_hits = np.zeros(10240)
for a, p in xyz[['a', 'pad']].values:
    exp_hits[int(p)] += a

In [ ]:
fig, sm = pad_plot(exp_hits, scale='linear', cmap='viridis')
plt.colorbar(sm)

In [ ]:
exp_mesh = np.zeros(512)
for z, a in raw_xyz[:, 2:4]:
    exp_mesh += transfer(a, 280./80, z)
    
res_mesh = fitter.evtgen.make_mesh_signal(res_track[:, :3].copy(), res_track[:, 4].copy())

In [ ]:
plt.plot(res_track[:, 2]*1000, res_track[:, 1]*1000, 'k', zorder=100)
plt.plot(xyz.w, xyz.v, '.')
plt.xlim(0, 1000)
plt.hlines(0, *plt.xlim(), color='g')
plt.ylim(-275, 275)
plt.gca().set_aspect(1)
plt.xlabel('z [mm]')
plt.ylabel('y [mm]')
plt.title('Run {:.0f} event {:.0f}'.format(fr.run_num, fr.evt_id))
# plt.savefig('/Users/josh/Desktop/plots_for_daniel/yz.pdf', bbox_inches='tight')

In [ ]:
plt.plot(res_track[:, 2]*1000, res_track[:, 0]*1000, 'k', zorder=100)
plt.plot(xyz.w, xyz.u, '.')
plt.xlim(0, 1000)
plt.hlines(0, *plt.xlim(), color='g')
plt.ylim(-275, 275)
plt.gca().set_aspect(1)
plt.xlabel('z [mm]')
plt.ylabel('x [mm]')
plt.title('Run {:.0f} event {:.0f}'.format(fr.run_num, fr.evt_id))
# plt.savefig('/Users/josh/Desktop/plots_for_daniel/xz.pdf', bbox_inches='tight')

In [ ]:
plt.plot(res_track[:, 0]*1000, res_track[:, 1]*1000, 'k', zorder=100)
plt.plot(xyz.u, xyz.v, '.')
plt.xlim(-275, 275)
plt.ylim(-275, 275)
plt.gca().set_aspect(1)
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
plt.title('Run {:.0f} event {:.0f}'.format(fr.run_num, fr.evt_id))
# plt.savefig('/Users/josh/Desktop/plots_for_daniel/xy.pdf', bbox_inches='tight')

In [ ]:
with sns.axes_style('ticks'), sns.plotting_context('paper'):
    plt.figure(figsize=(3.5, 3.5))
    plt.plot(res_track[:, 0]*1000, res_track[:, 1]*1000, 'k', zorder=100)
    plt.plot(xyz.u, xyz.v, '.')
    plt.gca().set_aspect(1)
    
    sns.despine()
    plt.xlabel('x [mm]')
    plt.ylabel('y [mm]')
    plt.tight_layout()
    
#     plt.savefig('/Users/josh/Documents/Papers/attpc-nim-paper/Figures/evt_{:04.0f}_{:0.0f}_xy.pdf'.format(fr.run_num, fr.evt_id))

In [ ]:
with sns.axes_style('ticks'), sns.plotting_context('paper'): 
    plt.figure(figsize=(3.5, 2.4))
    plt.plot(res_track[:, 2]*1000, res_track[:, 1]*1000, 'k', zorder=100)
    plt.plot(xyz.w, xyz.v, '.')
    
    sns.despine()
    plt.xlabel('z [mm]')
    plt.ylabel('y [mm]')
    plt.tight_layout()
    
    plt.savefig('/Users/josh/Documents/Papers/attpc-nim-paper/Figures/evt_{:04.0f}_{:0.0f}_zy.pdf'.format(fr.run_num, fr.evt_id))

In [ ]:
%matplotlib inline

In [ ]:
with sns.axes_style('white'):
    plt.subplot(111, projection='3d')
    plt.plot(res_track[:, 0]*1000, res_track[:, 1]*1000, res_track[:, 2]*1000, 'k')
    plt.plot((0, 0), (0, 0), (0, 1000), 'g')
    plt.gca().scatter(xyz.u, xyz.v, xyz.w, linewidths=0)
    plt.xlim(-275, 275)
    plt.ylim(-275, 275)
    plt.gca().set_zlim(0, 1000)
    plt.gca().set_aspect(1)
#     plt.savefig('/Users/josh/Desktop/plots_for_daniel/3d.pdf', bbox_inches='tight')

In [ ]:
fr

In [ ]:
np.rad2deg(fr.pol0) - 180